In [0]:
import requests
import base64
from databricks.sdk import WorkspaceClient
from dbruntime.databricks_repl_context import get_context

In [0]:
%sql
use catalog dkushari_uc;
use demodb;

In [0]:
%sql
drop table if exists test_tagging;

In [0]:
%sql
create table if not exists test_tagging(`account name` string, `account id?` string) TBLPROPERTIES (
    'delta.columnMapping.mode' = 'name'
  );


In [0]:
%sql
insert into test_tagging
values
  ('ANALYST_ES','1'),
  ('ANALYST_SPAIN','2');

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
select `account name`,`account id?`  from test_tagging;

account name,account id?
ANALYST_ES,1
ANALYST_SPAIN,2


In [0]:
def delete_tag(workspace_url, table_full_name, column_name, tag_key, headers):
    url = f"{workspace_url}/api/2.0/unity-catalog/tag-assignments/TABLE/{table_full_name}/COLUMN/{column_name}/{tag_key}"
    response = requests.delete(
        url,
        headers=headers
    )
    return response.status_code, response.json()

In [0]:
def add_tag(workspace_url, table_full_name, column_name, payload, headers):
    url = f"{workspace_url}api/2.0/unity-catalog/tag-assignments/TABLE/{table_full_name}/COLUMN/{column_name}"
    response = requests.post(url, headers=headers, json=payload)
    return response.status_code, response.json()

In [0]:
headers = {'Authorization' : 'Bearer {}'.format(dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()), "Content-Type": "application/json"}
workspace_url = "https://adb-984752964297111.11.azuredatabricks.net/"
workspace_url

'https://adb-984752964297111.11.azuredatabricks.net/'

In [0]:
w = WorkspaceClient()
table_full_name = "dkushari_uc.demodb.test_tagging"
column_name = "account name"
token = base64.b64decode(w.secrets.get_secret(scope="dkushari-scope", key="azure_pat_token").value).decode('utf-8')
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
tag_key = "pii_dk"
payload = {
    "tag_key": tag_key,
    "tag_value": "addr"
}
# Column with spaces
print("Example 1 -> Column with spaces")
response_code, response_json = add_tag(workspace_url, table_full_name, column_name, payload, headers)
print(f"Add Tag 1 : {response_json}")
if response_code != 200:
    if response_code == 409:
        response_code, response_json = delete_tag(workspace_url, table_full_name, column_name, tag_key, headers)
        print(f"Delete Tag : {response_json}")
        if response_code != 200:
            print(response_json)
        else:
            response_code, response_json = add_tag(workspace_url, table_full_name, column_name, payload, headers)
            print(f"Add Tag 2 : {response_json}")
            if response_code != 200:
                print(response_json)
# Column with spaces and ?
print("Example 2 -> Column with spaces and ?")
column_name = "account id%3F"
tag_key = "geo_region_dk"
payload = {
    "tag_key": tag_key
}
response_code, response_json = add_tag(workspace_url, table_full_name, column_name, payload, headers)
print(f"Add Tag 1 : {response_json}")
if response_code != 200:
    if response_code == 409:
        response_code, response_json = delete_tag(workspace_url, table_full_name, column_name, tag_key, headers)
        print(f"Delete Tag : {response_json}")
        if response_code != 200:
            print(response_json)
        else:
            response_code, response_json = add_tag(workspace_url, table_full_name, column_name, payload, headers)
            print(f"Add Tag 2 : {response_json}")
            if response_code != 200:
                print(response_json)

Example 1 -> Column with spaces
Add Tag 1 : {'tag_assignment': {'tag_key': 'pii_dk', 'tag_value': 'addr'}}
Example 2 -> Column with spaces and ?
Add Tag 1 : {'tag_assignment': {'tag_key': 'geo_region_dk', 'tag_value': ''}}


In [0]:
%sql
ALTER  TABLE test_tagging
ALTER COLUMN `account name`
SET TAGS('geo_region_dk');

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-457628674487877>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "ALTER  TABLE test_tagging\nALTER COLUMN `account name`\nSET TAGS('geo_region_dk');\n")

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2493, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2491 with self.builtin_trap:
   2492     args = (magic_arg_s, cell)
-> 2493     result = fn(*args, **kwargs)
   2495 # The code below prevents the output from being displayed
   2496 # when using magics with decorator @output_can_be_silenced
   2497 # when the last Python token in the expression is a ';'.
   2498 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:191, in SqlMagic.sql(self, line, cell)
    185     self.logger.logDriv

In [0]:
%sql
ALTER  TABLE test_tagging
ALTER COLUMN `account id%3F`
SET TAGS("pii_dk" = "addr");

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-457628674487937>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'ALTER  TABLE test_tagging\nALTER COLUMN `account id%3F`\nSET TAGS("pii_dk" = "addr");\n')

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2493, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2491 with self.builtin_trap:
   2492     args = (magic_arg_s, cell)
-> 2493     result = fn(*args, **kwargs)
   2495 # The code below prevents the output from being displayed
   2496 # when using magics with decorator @output_can_be_silenced
   2497 # when the last Python token in the expression is a ';'.
   2498 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:191, in SqlMagic.sql(self, line, cell)
    185     self.logger.logD